# Sélection des features

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:

pd.set_option("display.max_columns", None)
datas = pd.read_csv("Pélec_14_gaz_train.csv")
datas.head()

,OSEBuildingID,PrimaryPropertyType,CouncilDistrictCode,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuildings,ENERGYSTARScore,SiteEUIkBtusf,SiteEUIWNkBtusf,SourceEUIkBtusf,SourceEUIWNkBtusf,SiteEnergyUsekBtu,SiteEnergyUseWNkBtu,SteamUsekBtu,ElectricitykWh,ElectricitykBtu,NaturalGastherms,NaturalGaskBtu,OtherFuelUsekBtu,Outlier,TotalGHGEmissions,GHGEmissionsIntensity,SiteEUIkBtusf_diff1516,SiteEUIWNkBtusf_diff1516,SourceEUIkBtusf_diff1516,SourceEUIWNkBtusf_diff1516,SiteEnergyUsekBtu_diff1516,SiteEnergyUseWNkBtu_diff1516,SteamUsekBtu_diff1516,ElectricitykWh_diff1516,ElectricitykBtu_diff1516,NaturalGastherms_diff1516,NaturalGaskBtu_diff1516,TotalGHGEmissions_diff1516,GHGEmissionsIntensity_diff1516,GFAperFloor,GFAperBuilding,HasParking,percGFABuilding,percGFAParking,SteamperGFA,ElecperGFA,GasperGFA,FuelperGFA,percSteam,percElec,percGas,percFuel,WNfactor,SteamWN,ElecWN,GasWN,FuelWN,SteamWNperGFA,ElecWNperGFA,GasWNperGFA,FuelWNperGFA,percSteamWN,percElecWN,percGasWN,percFuelWN,WNfactor_diff,Steam_diff1516WN,Elec_diff1516WN,Gas_diff1516WN
0,1.0,hotel,7.0,1927.0,1.0,12.0,88434.0,0.0,88434.0,65.0,78.9,80.3,173.5,175.1,6981428.0,7097539.0,2023032.0,1080307.0,3686160.0,12724.0,1272388.0,0.0,NaN,249.43,2.64,2.799997,4.000003,9.0,13.9,244934.5,475482.0,0.0,76207.2500,259867.0,40.52930,4065.0,0.55,0.19,7369.500000,88434.0,0.0,1.000000,0.000000,22.876179,41.682611,14.387996,0.0,0.289773,0.527995,0.182253,0.0,0.983641,2.056678e+06,3.747466e+06,1.293550e+06,0.0,23.256642,42.375851,14.627288,0.0,0.289773,0.527995,0.182253,0.0,0.515129,0.000000e+00,504469.892539,7.891229e+03
1,2.0,hotel,7.0,1996.0,1.0,11.0,103566.0,15064.0,88502.0,51.0,94.4,99.0,191.3,195.2,8354235.0,8765788.0,0.0,1144563.0,3905411.0,44490.0,4448985.0,0.0,NaN,263.51,2.38,0.400003,0.000000,0.0,0.0,33698.0,310244.0,0.0,0.0000,0.0,6960.81641,696097.0,32.35,0.48,8045.636364,88502.0,1.0,0.854547,0.145453,0.000000,37.709393,42.957969,0.0,0.000000,0.467477,0.532542,0.0,0.953050,0.000000e+00,4.097802e+06,4.668154e+06,0.0,0.000000,39.567063,45.074199,0.0,0.000000,0.467477,0.532542,0.0,0.108618,0.000000e+00,0.000000,6.408686e+06
2,3.0,hotel,7.0,1969.0,1.0,41.0,961990.0,0.0,961990.0,18.0,96.6,99.7,242.7,246.5,73130656.0,75506272.0,19660404.0,14583930.0,49762435.0,37099.0,3709900.0,0.0,NaN,2061.48,1.92,0.000000,0.000000,0.0,0.0,0.0,806456.0,1906150.0,0.0000,0.0,0.00000,0.0,27.80,0.27,23463.170732,961990.0,0.0,1.000000,0.000000,20.437223,51.728641,3.856485,0.0,0.268839,0.680459,0.050730,0.0,0.968538,2.029906e+07,5.137895e+07,3.830414e+06,0.0,21.101117,53.409022,3.981761,0.0,0.268839,0.680459,0.050730,0.0,0.000000,1.326704e+07,0.000000,0.000000e+00
3,5.0,hotel,7.0,1926.0,1.0,10.0,61320.0,0.0,61320.0,1.0,460.4,462.5,636.3,643.2,28229320.0,28363444.0,23458518.0,811521.0,2769023.0,20019.0,2001894.0,0.0,high outlier,1936.34,31.38,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.3125,0.0,0.00000,0.0,0.00,0.00,6132.000000,61320.0,0.0,1.000000,0.000000,382.559002,45.156931,32.646673,0.0,0.830998,0.098090,0.070915,0.0,0.995271,2.356997e+07,2.782179e+06,2.011405e+06,0.0,384.376628,45.371482,32.801785,0.0,0.830998,0.098090,0.070915,0.0,0.000000,0.000000e+00,0.000000,0.000000e+00
4,8.0,hotel,7.0,1980.0,1.0,18.0,119890.0,12460.0,107430.0,67.0,120.1,122.1,228.8,227.1,14829099.0,15078243.0,0.0,1777841.0,6066245.0,87631.0,8763105.0,0.0,NaN,507.70,4.02,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,408.98438,40893.0,0.00,0.00,5968.333333,107430.0,1.0,0.896071,0.103929,0.000000,50.598424,73.092877,0.0,0.000000,0.409077,0.590940,0.0,0.983477,0.000000e+00,6.168164e+06,8.910334e+06,0.0,0.000000,51.448529,74.320912,0.0,0.000000,0.409077,0.590940,0.0,0.000000,0.000000e+00,0.000000,1.826739e+07


## classement des variables

* Les variables nominatives seront transformées par OneHotEncoder
* Les variables ordinales seront transformées par OrdinalEncoder
* Les variables numériques seront normalisées par QuantileTransformer

In [3]:
nom_features = ["PrimaryPropertyType", "CouncilDistrictCode", "Outlier"
    ]
ord_features = ["YearBuilt", "NumberofFloors", "NumberofBuildings", "HasParking" 
    ]
num_features = ["PropertyGFABuildings", "PropertyGFAParking", 
    "GFAperFloor", "GFAperBuilding", 
    "percGFABuilding", "percGFAParking", 
    
    # "SteamUsekBtu", "ElectricitykBtu", "NaturalGaskBtu", "OtherFuelUsekBtu", 
    "SteamUsekBtu_diff1516", "ElectricitykBtu_diff1516", "NaturalGaskBtu_diff1516", 

    "SteamperGFA", "ElecperGFA", "GasperGFA", "FuelperGFA", "percSteam", "percElec", "percGas", "percFuel", 

    "WNfactor", 
    # "SteamWN", "ElecWN", "GasWN", "FuelWN", 
    "SteamWNperGFA", "ElecWNperGFA", "GasWNperGFA", "FuelWNperGFA", 
    "percSteamWN", "percElecWN", "percGasWN", "percFuelWN", 
    "WNfactor_diff", "Steam_diff1516WN", "Elec_diff1516WN", "Gas_diff1516WN"
    ]

cible = ["TotalGHGEmissions"]

In [4]:
def to_log(X):
  return np.log(X + 1)

def to_exp(X):
  return np.exp(X) - 1

In [5]:
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer, OneHotEncoder, OrdinalEncoder, FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline

X = datas[nom_features + ord_features + num_features]
y = datas[cible]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

pipe = Pipeline([("qt", FunctionTransformer(func= to_log, inverse_func= to_exp)), ("scaler", StandardScaler())])

ct = ColumnTransformer([("onehotencoder", OneHotEncoder(handle_unknown="ignore"), nom_features),
    ("quantiletransformer", pipe, num_features),
    ("ordinalencoder", OrdinalEncoder(handle_unknown= "use_encoded_value", unknown_value= -1), ord_features)
    ])
ct.fit(X_train)

X_train_ct = pd.DataFrame(ct.transform(X_train), columns= list(ct.transformers_[0][1].get_feature_names(nom_features)) + 
    num_features + ord_features)
X_test_ct = pd.DataFrame(ct.transform(X_test), columns= list(ct.transformers_[0][1].get_feature_names(nom_features)) + num_features
    + ord_features)


In [6]:
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [7]:
from sklearn.feature_selection import RFECV
    
ttr = TransformedTargetRegressor(xgboost.XGBRegressor(objective= "reg:squarederror"), 
    transformer=QuantileTransformer(output_distribution="normal")
    )

rfe = RFECV(ttr, cv= 5, scoring= "r2", n_jobs= -1, importance_getter= "regressor_.feature_importances_")
rfe.fit(X_train_ct, y_train)


RFECV(cv=5,
      estimator=TransformedTargetRegressor(regressor=XGBRegressor(base_score=None,
                                                                  booster=None,
                                                                  colsample_bylevel=None,
                                                                  colsample_bynode=None,
                                                                  colsample_bytree=None,
                                                                  gamma=None,
                                                                  gpu_id=None,
                                                                  importance_type='gain',
                                                                  interaction_constraints=None,
                                                                  learning_rate=None,
                                                                  max_delta_step=None,
                                         

In [8]:
print(f"Train R2: {rfe.score(X_train_ct, y_train)}")
print(f"Test R2: {rfe.score(X_test_ct, y_test)}")

Train R2: 0.9997842511553728
Test R2: 0.9105343367726354


Bien que les R2 soient élevés, je ne considère pas qu'il y ait de l'overfitting dû au faible écart entre le R2 issu de train et celui issu de test

In [9]:
print(X_train_ct.columns[rfe.support_])

Index(['PrimaryPropertyType_hospital', 'PropertyGFABuildings',
       'GFAperBuilding', 'SteamperGFA', 'ElecperGFA', 'GasperGFA', 'percElec',
       'ElecWNperGFA'],
      dtype='object')


In [ ]:
pd.DataFrame(X_train_ct.columns[rfe.support_], columns=["columns"]).to_csv("Pélec_16_relevant_features.csv", index=False)
pd.DataFrame([], columns=["Ridge", "SVR", "LR", "Lasso", "XGB", "RFR"], index=["train_score", "test_score"]).to_csv("Pélec_16_scores.csv", 
    index=True)

In [ ]:
# files.download("Pélec_10_relevant_features.csv")
# files.download("Pélec_10_scores.csv")